# Requirements


In [ ]:
pip install openai
pip install git+https://github.com/openai/swarm.git

# Linkedin Agent Functions

In [45]:
import os
from google.colab import userdata
import requests
import json
import openai
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["Rapid_API_Key"] = userdata.get('Rapid_API')
os.environ["Email_password"] = userdata.get('Email_Password')


In [46]:
import requests

def profile_info(linkedin_url):
    querystring = {"url": linkedin_url}
    Rapid_API=userdata.get('Rapid_API')
    headers = {
        "x-rapidapi-key": Rapid_API,
        "x-rapidapi-host": "linkedin-data-api.p.rapidapi.com"
    }

    print("LinkedIn Agent called......")

    # Making the request
    try:
        response = requests.get(
            "https://linkedin-data-api.p.rapidapi.com/get-profile-data-by-url",
            headers=headers,
            params=querystring
        )

        response.raise_for_status()  # Raise an exception for HTTP errors

        # Try to parse the response as JSON
        data = response.json()

        # Fetch the last LinkedIn post
        last_linkedin_post = last_post(data['username'])

        # Prepare profile data
        profile_data = {
            "firstName": data.get('firstName', 'N/A'),
            "lastName": data.get('lastName', 'N/A'),
            "job_title": data.get('fullPositions', [{}])[0].get('title', 'N/A'),
            "job_company": data.get('fullPositions', [{}])[0].get('companyName', 'N/A'),
            "last_linkedin_post": last_linkedin_post,
        }

        print(profile_data)
        return profile_data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")
    except ValueError:  # This catches JSON decoding errors
        print("Error: Unable to decode JSON from response.")
        print("Response Text:", response.text)
    except KeyError as key_err:
        print(f"KeyError: {key_err}. Missing expected data in the response.")
    except Exception as e:
        print(f"An error occurred: {e}")


# Get profile's last post
def last_post(linkedin_username):
    print("Retreiving Last linkedin Post")
    querystring = {"username": linkedin_username}
    url = "https://linkedin-data-api.p.rapidapi.com/get-profile-posts"
    Rapid_API=userdata.get('Rapid_API')

    headers = {
        "x-rapidapi-key": Rapid_API,
        "x-rapidapi-host": "linkedin-data-api.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()  # Ensure the request was successful

        # Parse the response JSON
        data = response.json()

        data=data['data']
        # Get the last post if available

        last_post = data[0].get('text', 'No content available')


        print(last_post)
        return last_post

    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")
    except ValueError:
        print("Error: Unable to decode JSON from response.")
        print("Response Text:", response.text)
    except KeyError as key_err:
        print(f"KeyError: {key_err}. Missing expected data in the response.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Error retrieving last post"


# SEO Agent Functions

In [47]:
import os
import requests
import openai
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

def get_SEO_data(website_url):
    print("SEO Agent called......")

    # Securely load API key from environment variables
    Rapid_API=userdata.get('Rapid_API')
    headers = {
        "x-rapidapi-key": Rapid_API,
        "x-rapidapi-host": "ahrefs2.p.rapidapi.com"
    }

    try:
        ### Traffic Info
        url_traffic = "https://ahrefs2.p.rapidapi.com/traffic"
        querystring = {"url": website_url, "mode": "subdomains"}
        response_traffic = requests.get(url_traffic, headers=headers, params=querystring)
        response_traffic.raise_for_status()  # Raise an exception for bad responses
        traffic_data = response_traffic.json()

        ### Website Info
        url_authority = "https://ahrefs2.p.rapidapi.com/authority"
        response_website = requests.get(url_authority, headers=headers, params=querystring)
        response_website.raise_for_status()
        website_data = response_website.json()

        ### Extracting data for the report
        seo_params = {
            "traffic_history": (
                f"For date {traffic_data['traffic_history'][4]['date']}: "
                f"organic traffic was {traffic_data['traffic_history'][4]['organic']}, and "
                f"for date {traffic_data['traffic_history'][5]['date']}: "
                f"organic traffic was {traffic_data['traffic_history'][5]['organic']}"
            ),
            "website_data": (
                f"Domain Rating: {website_data['domainRating']}, "
                f"URL Rating: {website_data['urlRating']}, "
                f"Backlinks: {website_data['backlinks']}, "
                f"Referring Domains: {website_data['refdomains']}, "
                f"Dofollow Backlinks: {website_data['dofollowBacklinks']}, "
                f"Dofollow Referring Domains: {website_data['dofollowRefdomains']}"
            )
        }
        print(seo_params)
        # Generate SEO report using OpenAI
        seo_report = get_SEO_report(seo_params)
        return seo_report

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

def get_SEO_report(seo_params):
    client = openai.OpenAI()  # Ensure proper OpenAI client initialization
    prompt = f"""
    ### Role:
    You are an SEO Specialist.

    ### Task:
    Analyze the provided SEO data to uncover actionable insights that can enhance the website’s organic ranking.

    ### Context:
    You have access to traffic history, domain rating, URL rating, total backlinks, referring domains, and dofollow backlinks.
    Your goal is to evaluate these data points, identify patterns, and provide recommendations to improve the site’s organic visibility and overall SEO performance.

    ### Specific Instructions:
    - **Traffic History**:
      - Examine traffic trends over time, focusing on fluctuations between specific dates.
        Highlight any significant drops or spikes in organic traffic, pinpoint potential causes (e.g., algorithm updates, technical issues), and suggest corrective actions.

    - **Domain and URL Rating**:
      - Compare the domain rating and URL rating against competitors to evaluate the site’s authority.
        Identify areas to enhance authority through improved content quality, link-building strategies, or technical SEO optimizations.

    - **Backlink Profile**:
      - Assess the site’s backlink profile, including the total number of backlinks, referring domains, dofollow backlinks, and dofollow referring domains.
        Highlight strengths and weaknesses, and recommend strategies to acquire high-quality backlinks that could boost the site’s authority and rankings.

    ### Deliverables:
    - A comprehensive analysis of the site’s SEO performance, identifying key opportunities for improvement.
    - Actionable recommendations for enhancing organic search visibility and overall SEO health.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"Here is the required SEO data for generating SEO report: {seo_params}"
            },
            {
                "role": "system",
                "content": prompt
            }
        ]
    )

    # Access the content directly from the message object
    print(response.choices[0].message.content)
    return response.choices[0].message.content


# Email Composer Agent Functions

In [48]:
import openai

def compose_email_body(seo_report, linkedin_data):
    print("Email Writing Agent called")
    client = openai.OpenAI()
    # Initialize OpenAI API client
    openai.api_key = "your_openai_api_key"  # Ensure to replace with your actual API key

    # Compose the prompt with proper variable references
    prompt = f"""
    # Role:
    You are an experienced SEO specialist representing Webbografi. Your goal is to engage with a lead candidate from LinkedIn,
    you have access to their LinkedIn profile information: {linkedin_data}.
    You have access to an SEO report that highlights performance metrics and issues on their website: {seo_report}.

    # Task:
    Compose a brief yet personalized outreach email body, offering a complimentary SEO audit tailored to their website's needs.
    The audit will identify current issues and provide actionable steps to enhance search rankings, website performance, and user experience.

    ## Steps:
    1. Personalized Introduction:
        Start by addressing the lead's first name and reference a recent LinkedIn post to show familiarity with their work: use the following information : {linkedin_data}.

        Connect their post to the potential benefits of improving their website’s SEO.

    2. Problem Statement:
        Mention key areas of concern on their website based on the insights from the SEO report {seo_report}.

    3. Solution Offering:
        Describe how Webbografi's free SEO audit will uncover and diagnose these issues in detail, along with a step-by-step plan to resolve them.
        Highlight the audit’s value in improving their search engine rankings and overall website performance.

    # Specifics:
    Keep the email body concise, focusing on the key problem areas briefly. Make the offer clear and actionable.dont addsubject line
    """

    # Call the OpenAI API
    response =  client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": f"You are a helpful assistant skilled at writing professional cold emails."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    # Access the content directly from the message object
    print(response.choices[0].message.content)
    return response.choices[0].message.content




In [49]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email(receiver_email,body):


    client = openai.OpenAI()
    response =  client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[

            {
                "role": "user",
                "content": f"Generate a subject content  for an email based on the following body: {body}"

            }
        ]
    )
    print(response.choices[0].message.content)
    subject=response.choices[0].message.content
    # Email configuration
    sender_email = "sasmitshashwat@gmail.com"

    password = userdata.get('Email_Password')



    # Set up the MIME
    msg = MIMEMultipart()
    msg['From'] =sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach body to the email
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Server setup (for Gmail; change if using a different provider)
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Secure the connection
        server.login(sender_email, password)

        # Send the email
        text = msg.as_string()
        server.sendmail(sender_email, receiver_email, text)
        server.quit()

        print("Email sent successfully!")
        return "Email sent successfully!"
    except Exception as e:
        print(f"Failed to send email: {str(e)}")
        return f"Failed to send email: {str(e)}"




# Defining Agents


In [50]:
from swarm import Agent


# Function to transfer to SEO Agent
def transfer_to_SEO_Agent(website_url):
    """Hand off the website url for generating SEO report."""
    return SEO_agent

# Function to transfer to LinkedIn Agent
def transfer_to_Linkedin_Agent(linkedin_url):
    """Hand off the LinkedIn URL to get profile info."""
    return linkedin_agent

def transfer_to_Email_writing_Agent():
    """Hand off the data for composing the email"""
    return Email_writing_agent




linkedin_agent = Agent(
    name="Linkedin Agent",
    instructions="You are a helpful agent whose job is to fetch information from the LinkedIn profile of a person using their profile url",
    functions=[profile_info,transfer_to_SEO_Agent,],
)

SEO_agent=Agent(
    name="SEO Agent",
    instructions="you are a smart agent which provides SEO report for a website",
    functions=[get_SEO_data,transfer_to_Email_writing_Agent],
)
Email_writing_agent=Agent(
    name="Email Writing Agent",
    instructions="You are a smart agent tasked with composing and sending a cold email to a client offering a free SEO audit for their website. To make the email feel more personal and engaging, you use insights gathered from their LinkedIn profile.",
    functions=[compose_email_body,send_email],
)


user_interface_agent = Agent(
    name="User Interface Agent",
    instructions="You are a user interface agent that handles all interactions with the user. You need to always start with a objective for writing cold emails  for leads using their website URL and their linkedin-URL . Be concise.",
    functions=[transfer_to_Linkedin_Agent],
)


# Start Streaming



In [ ]:
from swarm.repl import run_demo_loop

if __name__ == "__main__":
    # Run the demo loop with the user interface agent
    run_demo_loop(user_interface_agent, stream=True)

Starting Swarm CLI 🐝
User: write a cold email email:sasmitshashwat@outlook.com website url:https://garyvaynerchuk.com/ linkedinUrl:https://www.linkedin.com/in/garyvaynerchuk/
User Interface Agent: transfer_to_Linkedin_Agent()
Linkedin Agent: profile_info()
Linkedin Agent: transfer_to_SEO_Agent()
LinkedIn Agent called......
Retreiving Last linkedin Post
Too many people are greatly underestimating the power of kindness.

Why? I think it’s because people have this idea that being nice somehow makes you weak.

They’re afraid that if they don’t bring down the hammer, people won’t respect them or they’ll get taken advantage of.

I want to change that idea.

Candor is not an excuse not to be kind…and being kind doesn’t prevent you from being an effective leader.
{'firstName': 'Gary', 'lastName': 'Vaynerchuk', 'job_title': 'Chairman of VaynerX, CEO of VaynerMedia', 'job_company': 'VaynerX ', 'last_linkedin_post': 'Too many people are greatly underestimating the power of kindness.\n\nWhy? I thi